In [12]:
import tensorflow as tf
from tensorflow import keras
import firebase_admin
from firebase_admin import ml
from firebase_admin import credentials
import tensorflow as tf


import numpy as np
import pandas as pd

# print tensorflow version
print(tf.__version__)

cred = credentials.Certificate(
    r'..\AddressStandardisation\streetstandardisation-firebase-adminsdk-d65t2-de0cb0478f.json')

data = pd.read_csv('./data.csv')

data.head()

# Extract street type from 'Street' column
data['Street Type'] = data['Street'].str.extract(r'(\b\w+$)', expand=True)

# get the features and labels
features = data['Non-Standardised Street']
labels = data['Street Type']

2.16.1


In [13]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
# split the data into train and test sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create the encoder
vectorizer = CountVectorizer(binary=True)

# Fit the encoder and transform the training data
train_features = vectorizer.fit_transform(train_features)

# Transform the testing data
test_features = vectorizer.transform(test_features)



# Create the encoder
label_encoder = LabelEncoder()

# Fit the encoder and transform the labels
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)

# Now you can one-hot encode the labels
train_labels = to_categorical(train_labels, num_classes=43)
test_labels = to_categorical(test_labels, num_classes=43)

KeyboardInterrupt: 

In [18]:
# Create the model
model = tf.keras.models.Sequential([
    keras.layers.Dense(
        16, input_dim=train_features.shape[1], activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(43, activation='softmax')  # Change this line
])

# create tflite model

# Compile the model
model.compile(loss='categorical_crossentropy',  # Change this line
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

# Train the model
model.fit(train_features, train_labels, epochs=10,
          validation_data=(test_features, test_labels))

# Evaluate the model
loss, accuracy = model.evaluate(test_features, test_labels)

print("Loss: ", loss)
print("Accuracy: ", accuracy)


IndexError: tuple index out of range

In [15]:
model.save('my_model.h5')


In [23]:

# load the my_model.h5
model = tf.keras.models.load_model('my_model.h5')

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Convert the model
tflite_model = converter.convert()

# Save the model.
with open('street_type_model.tflite', 'wb') as f:
    f.write(tflite_model)




AttributeError: 'Sequential' object has no attribute '_get_save_spec'